<a href="https://colab.research.google.com/github/zi2p/M-machine-learning/blob/main/M33001_%D0%9F%D0%B8%D1%81%D0%B0%D1%80%D0%B5%D0%B2%D0%B0_lab5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L) `women-clothing-accessories.csv`. Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация - процесс замены конфиденциального элемента данных на неконфиденциальный эквивалент.
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация - процесс приведения словоформы к лемме (её нормальной (словарной) форме).
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать гиперпараметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
drive.mount('/content/gdrive')
sms_spam = pd.read_csv('/content/gdrive/MyDrive/sms_spam.csv')

Mounted at /content/gdrive


In [ ]:
sms_spam.head()

,type,text
0,ham,Hope you are having a good week. Just checking in
1,ham,K..give back my thanks.
2,ham,Am also doing in cbe only. But have to pay.
3,spam,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,spam,okmail: Dear Dave this is your final notice to...


In [ ]:
sms_spam["type"] = sms_spam["type"].map({"ham" : 0, "spam" : 1})
sms_spam.head()

,type,text
0,0,Hope you are having a good week. Just checking in
1,0,K..give back my thanks.
2,0,Am also doing in cbe only. But have to pay.
3,1,"complimentary 4 STAR Ibiza Holiday or £10,000 ..."
4,1,okmail: Dear Dave this is your final notice to...


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk import tokenize                  # готовый токенизатор библиотеки nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer         # лемматизация

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
def start():
  wnl = WordNetLemmatizer()
  _stop_words = set(stopwords.words("english"))
  sms_spam["text"] = sms_spam["text"].str.lower()             # нижний регистр
  sms_spam["text"] = sms_spam["text"].apply(lambda x: ' '\
                    .join(wnl.lemmatize(t) for t in word_tokenize(x)))       # токенизация & лемматизация 
  return _stop_words

Данные обработаны, теперь приступим к созданию модели, векторизации и оценке.

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer  # модель "мешка слов"
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.naive_bayes import MultinomialNB  # наивный байесовский классификатор

In [ ]:
sms_spam["count"] = 0
for i in np.arange(0, len(sms_spam["text"])):
    sms_spam.loc[i, "count"] = len(sms_spam.loc[i, "text"])   

In [ ]:
sms_spam.head()

,type,text,count
0,0,Hope you are having a good week. Just checking in,49
1,0,K..give back my thanks.,23
2,0,Am also doing in cbe only. But have to pay.,43
3,1,"complimentary 4 STAR Ibiza Holiday or £10,000 ...",149
4,1,okmail: Dear Dave this is your final notice to...,161


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sms_spam["text"], sms_spam["type"])

In [ ]:
informations = []

In [ ]:
def t_vectorizer():  
  for first_index in range(1, 7):
    for second_index in range(first_index, 7):
      if (second_index != 1): 
        for max_features in range(1000, 10000, 5000):
          for max_df in np.arange(0.1, 0.5, 0.1):
            for min_df in np.arange(0, 0.01, 0.005):
              vectorizer = TfidfVectorizer(
                  ngram_range = (first_index, second_index),
                  max_features = max_features, 
                  min_df = min_df,
                  max_df = max_df,
                  analyzer = "word")
              v_X_train = vectorizer.fit_transform(X_train)
              mnb = MultinomialNB()
              mnb.fit(v_X_train, y_train)  
              test = vectorizer.transform(X_test)
              predict = mnb.predict(test)                 
              informations.append({
                  "vectorizer" : "TfidfVectorizer",
                  "ngram range" : f"({first_index},{second_index})",
                  "analyzer" : "words",
                  "min_df" : min_df,
                  "max_df" : max_df,
                  "max_features" : max_features,
                  "f1-score" : f1_score(predict, y_test),
                  "precision" : precision_score(predict, y_test),
                  "recall" : recall_score(predict, y_test),
                  "accuracy" : accuracy_score(predict, y_test)
                  })
                  
def c_vectorizer(analyzer):
  for first_index in range(1, 7):
    for second_index in range(first_index, 7):
      if (second_index != 1): 
        vectorizer = CountVectorizer(
            ngram_range = (first_index, second_index),
            stop_words = _stop_words,
            analyzer = analyzer)
        v_X_train = vectorizer.fit_transform(X_train)
        mnb = MultinomialNB()
        mnb.fit(v_X_train, y_train)  
        test = vectorizer.transform(X_test)
        predict = mnb.predict(test)
        informations.append({
            "vectorizer" : "CountVectorizer",
            "ngram range" : f"({first_index},{second_index})",
            "analyzer" : analyzer,
            "min_df" : -1,
            "max_df" : -1,
            "max_features" : -1,
            "f1-score" : f1_score(predict, y_test),
            "precision" : precision_score(predict, y_test),
            "recall" : recall_score(predict, y_test),
            "accuracy" : accuracy_score(predict, y_test)
            })

Время запускать программу и проверять ее на работоспособность.

In [ ]:
 nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
_stop_words = start()

In [ ]:
# t_vectorizer()

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to 

In [ ]:
c_vectorizer('word')
c_vectorizer('char')

In [ ]:
result = pd.DataFrame(informations)

In [ ]:
result.sort_values(by=["precision"], ascending=False).head(20)

,vectorizer,ngram range,analyzer,min_df,max_df,max_features,f1-score,precision,recall,accuracy
30,CountVectorizer,"(3,3)",char,-1,-1,-1,0.970822,0.958115,0.983871,0.992086
31,CountVectorizer,"(3,4)",char,-1,-1,-1,0.970822,0.958115,0.983871,0.992086
22,CountVectorizer,"(1,4)",char,-1,-1,-1,0.963158,0.958115,0.968254,0.989928
26,CountVectorizer,"(2,3)",char,-1,-1,-1,0.960630,0.958115,0.963158,0.989209
27,CountVectorizer,"(2,4)",char,-1,-1,-1,0.968254,0.958115,0.978610,0.991367
21,CountVectorizer,"(1,3)",char,-1,-1,-1,0.960630,0.958115,0.963158,0.989209
34,CountVectorizer,"(4,4)",char,-1,-1,-1,0.965517,0.952880,0.978495,0.990647
33,CountVectorizer,"(3,6)",char,-1,-1,-1,0.970667,0.952880,0.989130,0.992086
32,CountVectorizer,"(3,5)",char,-1,-1,-1,0.965517,0.952880,0.978495,0.990647
35,CountVectorizer,"(4,5)",char,-1,-1,-1,0.965517,0.952880,0.978495,0.990647


Получилось, что лучшим оказался `CountVectorizer` анализатор `char` с n-граммой `(3,3)`.

## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **а?** - ноль или один символ **а**
* **а+** - один или более символов **а**
* **а\*** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа


In [ ]:
result = re.findall('a?b.', 'aabbсabbcbb') 
print(result)

['abb', 'abb', 'bb']


In [ ]:
result = re.findall('a*b.', 'aabbсabbcbb') 
print(result)

['aabb', 'abb', 'bb']


In [ ]:
result = re.findall('a+b.', 'aabbсabbcbb') 
print(result)

['aabb', 'abb']


Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'abcdefghijkabcabcxabc') 
print(result)

['abcd', 'abca']


Вопрос на внимательность: почему нет abcx?

Скорее всего потому что был найден `abca`, а `abcx` пересекается с уже найденным значением.

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
text = "Токенизация, применяемая в области информационной безопасности, представляет собой "\
       "процесс замены конфиденциального элемента данных на неконфиденциальный эквивалент, "\
       "называемый токеном, который не имеет самостоятельного смысла/значения для внешнего или внутреннего использования." 
print(re.findall(r'\b\w{1,2}', text))

['То', 'пр', 'в', 'об', 'ин', 'бе', 'пр', 'со', 'пр', 'за', 'ко', 'эл', 'да', 'на', 'не', 'эк', 'на', 'то', 'ко', 'не', 'им', 'са', 'см', 'зн', 'дл', 'вн', 'ил', 'вн', 'ис']


### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
text = "Токенизация, применяемая в области информационной безопасности, представляет собой процесс"\
       " замены конфиденциального элемента данных на неконфиденциальный эквивалент, называемый токеном,"\
       " который не имеет самостоятельного смысла/значения для внешнего или внутреннего использования. "\
       "Токен — это ссылка (то есть идентификатор), которая сопоставляется с конфиденциальными данными "\
       "через систему токенизации. Сопоставление исходных данных с токеном использует методы, которые "\
       "делают невозможным обратное преобразование токенов в исходные данные вне системы токенизации, "\
       "например, с использованием токенов, созданных при помощи случайных чисел. Система токенизации "\
       "должна быть защищена и проверена на основе самых эффективных мер по обеспечению безопасности,"\
       " применяемых к защите конфиденциальных данных, хранению, аудиту, аутентификации и авторизации. "\
       "Система токенизации предоставляет приложениям обработки данных полномочия и интерфейсы для запроса "\
       "токенов или расшифровку конфиденциальных данных из токенов."

print(re.split(r"\.", text, maxsplit=2))       

['Токенизация, применяемая в области информационной безопасности, представляет собой процесс замены конфиденциального элемента данных на неконфиденциальный эквивалент, называемый токеном, который не имеет самостоятельного смысла/значения для внешнего или внутреннего использования', ' Токен — это ссылка (то есть идентификатор), которая сопоставляется с конфиденциальными данными через систему токенизации', ' Сопоставление исходных данных с токеном использует методы, которые делают невозможным обратное преобразование токенов в исходные данные вне системы токенизации, например, с использованием токенов, созданных при помощи случайных чисел. Система токенизации должна быть защищена и проверена на основе самых эффективных мер по обеспечению безопасности, применяемых к защите конфиденциальных данных, хранению, аудиту, аутентификации и авторизации. Система токенизации предоставляет приложениям обработки данных полномочия и интерфейсы для запроса токенов или расшифровку конфиденциальных данных 

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print(result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
text = "Alex Rofle «Падение и рост токенизации», Май 13, 2015"
print(re.sub('\\d', 'DIG', text))

Alex Rofle «Падение и рост токенизации», Май DIGDIG, DIGDIGDIGDIG


**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
text = "Предыдущие и последующие тексты были взяты с https://ru.wikipedia.org/wiki/%D0%A2%D0%BE%D0%BA%D0%B5%D0%BD%D0%B8%D0%B7%D0%B0%D1%86%D0%B8%D1%8F_(%D0%B8%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%86%D0%B8%D0%BE%D0%BD%D0%BD%D0%B0%D1%8F_%D0%B1%D0%B5%D0%B7%D0%BE%D0%BF%D0%B0%D1%81%D0%BD%D0%BE%D1%81%D1%82%D1%8C"
print(re.sub(r'\b\S+://\S+\b', '', text))

Предыдущие и последующие тексты были взяты с 


### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog_01 = re.compile(r'[А-Яа-яё\-]{4,}')
prog_01.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'больше', 'больше', 'слов', 'Что-то']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
prog_02 = re.compile(r'@\S+\b')
prog_02.findall("abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz")

['@gmail.com', '@test.in', '@analyticsvidhya.com', '@rest.biz']